In [ ]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import time

from langgraph.checkpoint.memory import InMemorySaver

In [3]:
# State
class CrashState(TypedDict):

  input : str
  step1 : str
  step2 : str
  step3 : str


In [19]:
# Making All Nodes

def step1(state : CrashState):
  print("Step1 is Runing")
  time.sleep(2)
  state['step1'] = "Step1 is Done"

  return state

def step2(state : CrashState):
  print("Step2 is Hanging ... now manually interrupt from the notebook toolbar (STOP BUTTON)")
  time.sleep(20)

  return {'step2': 'Step2 is Done'}

def step3(state : CrashState):
  print("Step3 is Runing")
  return {'step3': 'Done step3 and NOW END'}

In [20]:
# Building Graph
graph = StateGraph(CrashState)

# MAking Node
graph.add_node('step_1', step1)
graph.add_node('step_2', step2)
graph.add_node('step_3', step3)

# Making Ebdes
graph.add_edge(START, 'step_1')
graph.add_edge('step_1', 'step_2')
graph.add_edge('step_2', 'step_3')
graph.add_edge('step_3', END)

# Familer With Persistence
checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [22]:
try:
  print("Start graph properly")
  config1 = {'configurable' : {'thread_id' : 'thread-1'}}
  workflow.invoke(
      {'input': 'START'},
      config = config1
  )
except:
  print("Kernel manually interrupt (CRASH Simulated)")

Start graph properly
Step1 is Runing
Step2 is Hanging ... now manually interrupt from the notebook toolbar (STOP BUTTON)
Kernel manually interrupt (CRASH Simulated)


In [23]:
# Now I want to see current state
workflow.get_state(config=config1)

StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-b79a-6d20-8004-eba2faedde84'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-01-14T02:12:04.450390+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-a47d-648f-8003-24c8e1b13dc5'}}, tasks=(PregelTask(id='cb96461c-3c74-b964-8c80-96df2a33d19b', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [24]:
# Now I want to check STATE history and running status
workflow.get_state_history(config=config1)

<generator object Pregel.get_state_history at 0x7c9ec7fbd0c0>

In [25]:
history = list(workflow.get_state_history(config=config1))
for state in history:
    print(state)


StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-b79a-6d20-8004-eba2faedde84'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2026-01-14T02:12:04.450390+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-a47d-648f-8003-24c8e1b13dc5'}}, tasks=(PregelTask(id='cb96461c-3c74-b964-8c80-96df2a33d19b', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())
StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-a47d-648f-8003-24c8e1b13dc5'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-01-14T02:12:02.446021+00:00', parent_config={'configurable': {'thread_id': 'thr

### Rerun this from where it was interrupt, means from step-2

In [26]:
workflow.invoke(None, config=config1)

Step2 is Hanging ... now manually interrupt from the notebook toolbar (STOP BUTTON)
Step3 is Runing


{'input': 'START',
 'step1': 'Step1 is Done',
 'step2': 'Step2 is Done',
 'step3': 'Done step3 and NOW END'}

In [27]:
# Current State
workflow.get_state(config1)

StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done', 'step2': 'Step2 is Done', 'step3': 'Done step3 and NOW END'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee8-0370-654a-8006-c21769e76245'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2026-01-14T02:12:39.245728+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee8-036c-641a-8005-19a10baca1ae'}}, tasks=(), interrupts=())

In [28]:
history = list(workflow.get_state_history(config=config1))

for state in history:
    print(state)

StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done', 'step2': 'Step2 is Done', 'step3': 'Done step3 and NOW END'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee8-0370-654a-8006-c21769e76245'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2026-01-14T02:12:39.245728+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee8-036c-641a-8005-19a10baca1ae'}}, tasks=(), interrupts=())
StateSnapshot(values={'input': 'START', 'step1': 'Step1 is Done', 'step2': 'Step2 is Done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee8-036c-641a-8005-19a10baca1ae'}}, metadata={'source': 'loop', 'step': 5, 'parents': {}}, created_at='2026-01-14T02:12:39.244037+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0ee6-b79a-6d20-8004-eba2fa